In [1]:
import pandas as pd
from pprint import pprint
import folium 
import gpxpy
import os

c:\ProgramData\Anaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
nodes_df = pd.read_csv ('lookout_trail_nodes.csv')
segments_df = pd.read_csv('lookout_trail_segments.csv')
nodes_df

,parent_node,elevation,connect_1,distance_1,connect_2,distance_2,connect_3,distance_3,connect_4,distance_4,connect_5,distance_5,parking
0,1,748.03,2,1.1496,8,0.79,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,2,692.26,1,1.1496,3,0.51,8.0,0.42,NaN,NaN,NaN,NaN,1.0
2,3,675.85,2,0.5089,4,1.00,5.0,1.14,NaN,NaN,NaN,NaN,NaN
3,4,846.46,3,1.0000,5,0.47,6.0,0.52,7.0,0.74,NaN,NaN,1.0
4,5,738.19,3,1.1400,4,0.47,6.0,0.70,NaN,NaN,NaN,NaN,NaN
5,6,935.04,4,0.5200,5,0.70,7.0,1.19,NaN,NaN,NaN,NaN,NaN
6,7,967.85,4,0.7400,6,1.19,8.0,0.11,9.0,0.39,24.0,0.86,NaN
7,8,990.81,1,0.7900,2,0.42,7.0,0.11,NaN,NaN,NaN,NaN,NaN
8,9,295.93,7,0.3900,10,0.15,11.0,0.61,NaN,NaN,NaN,NaN,NaN
9,10,315.62,9,0.1500,24,0.75,21.0,0.40,NaN,NaN,NaN,NaN,NaN


In [3]:
node_list = []
node_elevation = []
for i in range(0, len(nodes_df)):
    node_row = nodes_df.iloc[i]
    node_num = int(node_row['parent_node'])
    connect_list = []
    connect_1 = [int(node_row['connect_1']), node_row['distance_1']]
    connect_list.append(connect_1)
    connect_2 = [int(node_row['connect_2']), node_row['distance_2']]
    connect_list.append(connect_2)
    if not pd.isna(node_row['connect_3']):
        connect_3 = [int(node_row['connect_3']), node_row['distance_3']]
        connect_list.append(connect_3)
    if not pd.isna(node_row['connect_4']):
        connect_4 = [int(node_row['connect_4']), node_row['distance_4']]
        connect_list.append(connect_4)
    if not pd.isna(node_row['connect_5']):
        connect_5 = [int(node_row['connect_5']), node_row['distance_5']]
        connect_list.append(connect_5)
    node_list.append(connect_list)
    node_elevation.append(int(node_row['elevation']))
node_elevation

[748,
 692,
 675,
 846,
 738,
 935,
 967,
 990,
 295,
 315,
 824,
 781,
 696,
 204,
 685,
 764,
 473,
 925,
 820,
 919,
 397,
 535,
 538,
 1197]

In [4]:
def find_nodes(working_list, node_list):
    new_working_list = []
    for path in working_list:
        last_node = path['node_list'][-1]['node']
        next_nodes = node_list[last_node-1]
        segments = list(map(lambda n: n['segment'], path['node_list']))
        
        for node in next_nodes:
            s = [node[0], last_node]
            s.sort()
            segment = "s{:02d}{:02d}".format(s[0], s[1])
            if segment in segments:
                continue
            formatted_node = {
                'node': node[0],
                'new_distance': node[1],
                'segment': segment
            }
            new_path = {
                'total_distance': path['total_distance'] + node[1],
                'node_list': path['node_list'] + [formatted_node]
            }
            new_working_list.append(new_path)
    return new_working_list
            
def check_nodes(working_list, final_list, max_distance, min_distance):
    new_working_list = []
    start_node = working_list[0]['node_list'][0]['node']
    for path in working_list:
        if path['node_list'][-1]['node'] == start_node and \
                   path['total_distance'] < max_distance and \
                   path['total_distance'] > min_distance:
            unique = True
            segments = list(n['segment'] for n in path['node_list'])
            for final_path in final_list:
                final_segments = list(n['segment'] for n in final_path['node_list'])
                if sorted(segments) == sorted(final_segments):
                    unique = False
            if unique:
                final_list.append(path)
#             nodes = []
#             for node in path['node_list']:
#                 nodes.append(node['node'])
#             print('distance: {:.2f}, nodes: {}'.format(path['total_distance'], nodes))
            continue
        if path['total_distance'] < max_distance:
            new_working_list.append(path)
    return [new_working_list, final_list]


In [5]:
def get_paths(start, max_distance, min_distance):
    start_node = {
        'node': start,
        'new_distance': 0,
        'segment': ''
    }

    start_path = {
        'total_distance': 0,
        'node_list': [start_node]
    }

    working_list = [start_path]
    final_list = []

    while len(working_list) > 0:
    # for i in range(0, 4):
        working_list = find_nodes(working_list, node_list)
        checked_lists = check_nodes(working_list, final_list, max_distance, min_distance)
        working_list = checked_lists[0]
        final_list = checked_lists[1]
        # print('Acceptable Paths: {}, Working List: {}'.format(len(final_list), len(working_list)))
    return(final_list)
    



In [6]:
def draw_map(start, max_distance, min_distance, path_num):

    final_list = get_paths(start, max_distance, min_distance)
    print("Acceptable Paths: {}".format(len(final_list)))

    paths = []
    for path in final_list:
        segments = list(n['segment'] for n in path['node_list'])
        segments.pop(0)
        paths.append(segments)

    distance = final_list[path_num]['total_distance']
    segs = paths[path_num]
    print('Current Length: {}, Current Segment List: {}'.format(distance, segs))
    zoom = 14
    lat = 35.005
    long = -85.344952
    myMap = folium.Map(location=[lat,long],zoom_start=zoom)

    for seg in segs:
        filename = seg + '.gpx'
        gpx_file = open(filename, 'r')
        gpx = gpxpy.parse(gpx_file)
        points = []
        for track in gpx.tracks:
            for segment in track.segments:        
                for point in segment.points:
                    points.append(tuple([point.latitude, point.longitude]))
        folium.PolyLine(points, color="blue", weight=2.5, opacity=1).add_to(myMap)
    myMap.fit_bounds(myMap.get_bounds())
    
    return(myMap)

In [8]:
# fix s0724
draw_map(23,6,4,1)

Acceptable Paths: 17
Current Length: 4.590000000000001, Current Segment List: ['s1323', 's1213', 's1112', 's1119', 's1920', 's2023']
